In [53]:
%pip install -q tranqu

Note: you may need to restart the kernel to use updated packages.


In [54]:
from qubex.circuit.simulator import SimulatorBackend
from tranqu import Tranqu

In [30]:
oqtopus_device ={
  "name": "1",
  "qubits": [
    {
      "id": 0,
      "physical_id": 5,
      "position": {
        "x": 3,
        "y": 0
      },
      "fidelity": 0.0,
      "meas_error": {
        "prob_meas1_prep0": 0.0,
        "prob_meas0_prep1": 0.0
      },
      "qubit_lifetime": {
        "t1": 0.0,
        "t2": 0.0
      },
      "gate_duration": {
        "rz": 0,
        "sx": 0,
        "x": 0
      }
    },
    {
      "id": 1,
      "physical_id": 7,
      "position": {
        "x": 3,
        "y": -1
      },
      "fidelity": 0.0,
      "meas_error": {
        "prob_meas1_prep0": 0.0,
        "prob_meas0_prep1": 0.0
      },
      "qubit_lifetime": {
        "t1": 0.0,
        "t2": 0.0
      },
      "gate_duration": {
        "rz": 0,
        "sx": 0,
        "x": 0
      }
    }
  ],
  "couplings": [
    {
      "control": 0,
      "target": 1,
      "fidelity": 0.0,
      "gate_duration": {
        "cx": 0,
        "rzx90": 0
      }
    }
  ],
  "timestamp": "2025-02-06 23:34:15.157724"
}

In [52]:
program = """OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;

h q[0];
cx q[0], q[1];
"""
options = {
    "basis_gates": [ "sx", "x", "rz", "cx"]
}

tranqu = Tranqu()
result = tranqu.transpile(
    program=program,
    program_lib="openqasm3",
    transpiler_lib="ouqu-tp",
    transpiler_options=options,
    device=oqtopus_device,
    device_lib="oqtopus",
)

Traceback (most recent call last):
  File "/home/edge/work/qubex/.venv/bin/ouqu-tp", line 8, in <module>
    sys.exit(main())
  File "/home/edge/work/qubex/.venv/lib/python3.10/site-packages/ouqu_tp/cli/main.py", line 43, in main
    raise Exception(
Exception: [ERROR] staq seems to be not installed.
please install staq from https://github.com/softwareQinc/staq
Traceback (most recent call last):
  File "/home/edge/work/qubex/.venv/lib/python3.10/site-packages/ouqu_tp/servicers/ouqu_tp.py", line 35, in transpile
    result, mapping = ouqu(qasm, cn_json)
  File "/home/edge/work/qubex/.venv/lib/python3.10/site-packages/ouqu_tp/servicers/exec_ouqu.py", line 185, in ouqu
    result = exec_ouqu(qasm_str, device_topology_json)
  File "/home/edge/work/qubex/.venv/lib/python3.10/site-packages/ouqu_tp/servicers/exec_ouqu.py", line 164, in exec_ouqu
    raise Exception(subprocess_output.stderr)
Exception: Traceback (most recent call last):
  File "/home/edge/work/qubex/.venv/bin/ouqu-tp", line 8,

AttributeError: 'NoneType' object has no attribute 'line'

In [48]:
def virtual_physical_map(device_topology_json: dict):
	device_topology = device_topology_json
	qubits = {
		qubit["id"]: qubit["physical_id"] for qubit in device_topology["qubits"]
	}
	couplings = {
		(c["control"], c["target"]): (qubits[c["control"]], qubits[c["target"]])
		for c in device_topology["couplings"]
	}
	return {"qubits": qubits, "couplings": couplings}

In [49]:
_virtual_physical_map = virtual_physical_map(oqtopus_device)
print(_virtual_physical_map)

{'qubits': {0: 5, 1: 7}, 'couplings': {(0, 1): (5, 7)}}


In [50]:
qasm_program = result.transpiled_program
print(qasm_program)

OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
rz(pi/2) q[0];
sx q[0];
rz(pi/2) q[0];
cx q[0], q[1];



In [44]:
backend = SimulatorBackend(virtual_physical_map=_virtual_physical_map)
backend.load_program(qasm_program)
print(backend.program)


OPENQASM 3.0;
include "stdgates.inc";
rz(pi/2) $0;
sx $0;
rz(pi/2) $0;
cx $0, $1;



In [45]:
backend.compile()

CircuitInstruction(operation=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[1.5707963267948966]), qubits=(<qiskit.circuit.quantumregister.Qubit object at 0x7f4b851764c0>,), clbits=())
None


KeyError: None

In [46]:
qasm_program = """
OPENQASM 3;
include "stdgates.inc";
qubit[2] q;
bit[2] c;
rz(1.5708) q[0];
sx q[0];
rz(1.5708) q[0];
cx q[0], q[1];
c[0] = measure q[0];
c[1] = measure q[1];
"""
backend.load_program(qasm_program)
backend.compile()

CircuitInstruction(operation=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[1.5708]), qubits=(Qubit(QuantumRegister(2, 'q'), 0),), clbits=())
Qubit(QuantumRegister(2, 'q'), 0)
0
CircuitInstruction(operation=Instruction(name='sx', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0),), clbits=())
Qubit(QuantumRegister(2, 'q'), 0)
0
CircuitInstruction(operation=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[1.5708]), qubits=(Qubit(QuantumRegister(2, 'q'), 0),), clbits=())
Qubit(QuantumRegister(2, 'q'), 0)
0
CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)), clbits=())
Qubit(QuantumRegister(2, 'q'), 0)
0
CircuitInstruction(operation=Instruction(name='measure', num_qubits=1, num_clbits=1, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0),), clbits=(Clbit(ClassicalRegister(2, 'c'), 0),))
Qubit(QuantumRegister(2, 'q')

In [78]:
backend.qubits

['Q05', 'Q07']

In [26]:
backend.couplings

['Q05-Q07', 'Q07-Q05']

In [27]:
backend.virtual_to_physical

{0: 'Q05', 1: 'Q07'}

In [28]:
backend.physical_to_virtual

{'Q05': 0, 'Q07': 1}

In [29]:
result = backend.execute()

In [30]:
result

{'00': 510, '11': 514}

In [7]:
from qubex.circuit.dummy import DummyBackend
nodes = ["Q05","Q07"]
edges =  ["Q05-Q07"]
backend = DummyBackend(chip_id="64Q", nodes=nodes, edges=edges)

qasm_program = """
OPENQASM 3;
include "stdgates.inc";
qubit[2] q;
bit[2] c;
rz(1.5708) q[0];
sx q[0];
rz(1.5708) q[0];
cx q[0], q[1];
c[0] = measure q[0];
c[1] = measure q[1];
"""

backend.load_program(qasm_program)
backend.execute()

{'00': 455, '01': 0, '10': 0, '11': 545}

In [ ]:
result

In [ ]:
result = backend.execute()

In [ ]:
result.probabilities

In [ ]:
backend.virtual_to_physical

In [ ]:
result.probabilities